In [1]:
import pandas as pd
import numpy as np

import datetime 
import pickle

import openpyxl


In [2]:
data = pd.read_csv("Daten_Aufgabe_1 - Shopauswertung.csv", delimiter=';')
data.head(1)


,Datum,Produkt,Umsatz
0,01.01.2021,A,17.132115


In [3]:
data.to_excel("Shop_eu.xlsx", engine='openpyxl', float_format='%.2f')

In [4]:
data = pd.read_excel("Daten_eu_sauber.xlsx", sheet_name="SAUBER")
data.head(1)

,Datum,Produkt,Umsatz
0,01.01.2021,A,17.13


In [5]:
data['Datum'] = pd.to_datetime(data['Datum'], format='%d.%m.%Y')
data['Monat'] = data['Datum'].dt.strftime('%B')
data.head(1)


,Datum,Produkt,Umsatz,Monat
0,2021-01-01,A,17.13,January


In [6]:
data_grouped = data.groupby(['Monat', 'Produkt'])['Umsatz'].sum().reset_index()

data_pivot =data_grouped.pivot(index='Monat', columns='Produkt', values='Umsatz')
data_pivot.columns = ['Umsatz A', 'Umsatz A+B+C','Umsatz B']

data_pivot.head(3)


,Umsatz A,Umsatz A+B+C,Umsatz B
Monat,,,
April,1392.18,1097119.01,79808.05
August,1587.99,2053853.51,177628.52
December,1603.26,2991358.45,274277.52


In [7]:
data_pivot["Umsatz C"] = data_pivot["Umsatz A+B+C"] - (data_pivot["Umsatz B"] + data_pivot["Umsatz A"])

data_pivot["Umsatz A"] = round(data_pivot["Umsatz A"],2)
data_pivot["Umsatz B"] = round(data_pivot["Umsatz B"],2)
data_pivot["Umsatz C"] = round(data_pivot["Umsatz C"],2)
data_pivot["Umsatz A+B+C"] = round(data_pivot["Umsatz A+B+C"],2)

data_pivot.head(3)

,Umsatz A,Umsatz A+B+C,Umsatz B,Umsatz C
Monat,,,,
April,1392.18,1097119.01,79808.05,1015918.78
August,1587.99,2053853.51,177628.52,1874637.00
December,1603.26,2991358.45,274277.52,2715477.67


In [8]:
data_pivot.reset_index(inplace=True)

In [9]:
data_pivot["Monat"] = pd.Categorical(data_pivot["Monat"],
                    categories=["January", "February", "March",
                    "April", "May", "June", "July", "August",
                    "September", "October", "November", "December"], ordered=True)
data_pivot = data_pivot.sort_values(by=['Monat'])


In [10]:
data_pivot.head(3)

,Monat,Umsatz A,Umsatz A+B+C,Umsatz B,Umsatz C
4,January,1514.89,425701.69,15656.52,408530.28
3,February,1495.04,590579.79,35189.58,553895.17
7,March,1788.99,881726.71,60515.05,819422.67


In [11]:
data_pivot.index = pd.RangeIndex.from_range(range(0, 12))

In [12]:
shop_eu = data_pivot[["Monat", "Umsatz A", "Umsatz B", "Umsatz C", "Umsatz A+B+C"]]
#data_pivot.head(3)
shop_eu.head(3)

,Monat,Umsatz A,Umsatz B,Umsatz C,Umsatz A+B+C
0,January,1514.89,15656.52,408530.28,425701.69
1,February,1495.04,35189.58,553895.17,590579.79
2,March,1788.99,60515.05,819422.67,881726.71


In [13]:
shop_eu = shop_eu.rename(columns={"Umsatz A":"Produkt_A_EU", "Umsatz B":"Produkt_B_EU", "Umsatz C":"Produkt_C_EU",
"Umsatz A+B+C":"Gesamt_EU"})

In [14]:
shop_eu.to_pickle("shop_eu.pkl")
shop_EU = pd.read_pickle("shop_eu.pkl")
shop_EU

,Monat,Produkt_A_EU,Produkt_B_EU,Produkt_C_EU,Gesamt_EU
0,January,1514.89,15656.52,408530.28,425701.69
1,February,1495.04,35189.58,553895.17,590579.79
2,March,1788.99,60515.05,819422.67,881726.71
3,April,1392.18,79808.05,1015918.78,1097119.01
4,May,1719.59,104693.52,1257335.38,1363748.49
5,June,1588.10,122367.05,1425203.06,1549158.21
6,July,1813.78,151031.05,1650579.04,1803423.87
7,August,1587.99,177628.52,1874637.00,2053853.51
8,September,1495.50,198167.05,2000719.74,2200382.29
9,October,1836.12,227776.05,2296581.83,2526194.00
